# Query data based on multiple conditions

## Package imports

In [ ]:
import sys

sys.path.append("..")
from pytreedb import db

## Import database

Define (local) MongoDB connection and import database from URL

In [ ]:
mydbfile = "my_complex_query_pytree.db"
db_url = "https://github.com/3dgeo-heidelberg/pytreedb/raw/main/data/test/data.db"
mydb = db.PyTreeDB(dbfile=mydbfile)
mydb.import_db(db_url, overwrite=True)

## Queries with logical operator between conditions

Get all trees for which TLS point clouds were acquired **AND** point clouds of quality 1 (best quality) are available (acquired from **any** platform). 

In [ ]:
len(mydb.query({"$and": [{"properties.data.quality": 1}, {"properties.data.mode": "TLS"}]}))

We can also include numeric comparison in our conditions (see `04_query_on_single_fields_and_download_laz`).

Get all trees for which TLS point clouds were acquired **AND these TLS point clouds** are of quality **grade 2 or better**. Becausee multiple conditions should be fulfilled within one element of an array (here: "data"), we have to use the `"$elemMatch"` operator.

In [ ]:
len(mydb.query({"properties.data": {"$elemMatch": {"quality": {"$lte": 2}, "mode": "TLS"}}}))

We can use a number of conditions and connect them with logical operators (e.g., `$or` and `$and`).

Get all tress which were acquired with ULS **AND NOT** in leaf-off condition (no matter from which platform). 

In [ ]:
len(
    mydb.query(
        {
            "$and": [
                {"properties.data.mode": "ULS"},
                {"properties.data.canopy_condition": {"$not": {"$regex": "^leaf-off"}}},
            ]
        },
    )
)

Get all tress which were **NOT** acquired with ULS **AND NOT** ever in leaf-off condition. 

In [ ]:
len(
    mydb.query(
        {
            "$and": [
                {"properties.data.mode": {"$not": {"$regex": "^ULS"}}},
                {"properties.data.canopy_condition": {"$not": {"$regex": "^leaf-off"}}},
            ]
        },
    )
)

Get all trees for which leaf-off ULS data **OR** TLS data is available.

In [ ]:
len(
    mydb.query(
        {
            "$or": [
                {"properties.data": {"$elemMatch": {"canopy_condition": "leaf-off", "mode": "ULS"}}},
                {"properties.data.mode": "TLS"},
            ]
        },
    )
)

Get all trees for which leaf-off ULS data **OR** TLS data is available **AND** which are taller than 20 m.

In [ ]:
len(
    mydb.query(
        {
            "$and": [
                {
                    "$or": [
                        {"properties.data": {"$elemMatch": {"canopy_condition": "leaf-off", "mode": "ULS"}}},
                        {"properties.data.mode": "TLS"},
                    ]
                },
                {"properties.measurements.height_m": {"$gt": 20}},
            ]
        }
    )
)

Get all trees for which neither field measurements **NOR** TLS measurements are available **AND** which are taller than 20 m. 

In [ ]:
len(
    mydb.query(
        {
            "$and": [
                {"$nor": [{"properties.measurements.source": "FI"}, {"properties.measurements.source": "TLS"}]},
                {"properties.measurements.height_m": {"$gt": 20}},
            ]
        }
    )
)